In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
!pip install -U keras
!pip install tensorflow-gpu==2.1.0rc0

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)
     |████████████████████████████████| 402.3MB 36kB/s 
     |████████████████████████████████| 450kB 46.8MB/s 
     |████████████████████████████████| 3.8MB 39.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=4c63d6fbcf866b720f359defefd6bc11ecb805b42dee7454427c2bd2727fd755
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which

In [3]:
import math
import random
import keras
from keras.layers import *
from keras.models import Sequential
from keras import Model
from keras import backend as K  
from keras.regularizers import l2
from sklearn import metrics

Using TensorFlow backend.


In [0]:
PRED_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/PR/KFold LadderNet/"


In [5]:
!ls "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/PR/KFold LadderNet/"


'Fold 0'  'Fold 5'			  'Object-wise Metrics Train.npy'
'Fold 1'  'Fold 6'			  'Pixel-wise Metrics Test.npy'
'Fold 2'  'Fold 7'			  'Pixel-wise Metrics Train.npy'
'Fold 3'  'Fold 8'			   PredY.npy
'Fold 4'  'Object-wise Metrics Test.npy'   TrueY.npy


In [8]:
test = np.load(PRED_PATH+'Pixel-wise Metrics Test.npy')
print(test)

[[[0.99588784 0.9998027  0.99784143]
  [0.         1.         0.        ]
  [0.55941101 0.02477235 0.04744375]
  [0.         1.         0.        ]]

 [[0.99997692 0.99999989 0.99998841]
  [1.         1.         1.        ]
  [1.         0.         0.        ]
  [0.32406288 0.60496614 0.42204724]]

 [[0.98567067 0.88777449 0.93416482]
  [0.23372464 0.24553069 0.23948225]
  [0.27094441 0.21965863 0.24262091]
  [0.61245133 0.97622333 0.75268939]]

 [[0.99715647 0.99282369 0.99498536]
  [0.37755323 0.71255168 0.49357853]
  [0.23584681 0.26933683 0.25148175]
  [0.76978417 0.00101842 0.00203414]]

 [[0.99977665 0.99928651 0.99953152]
  [0.08088798 0.37657401 0.1331709 ]
  [0.         1.         0.        ]
  [0.58715596 0.08705791 0.1516331 ]]

 [[0.99989107 0.99977894 0.999835  ]
  [0.         0.                nan]
  [0.         1.         0.        ]
  [0.55555556 0.01708428 0.03314917]]

 [[1.         0.99741131 0.99870398]
  [0.         1.         0.        ]
  [0.         1.         0

In [32]:
def tversky(y_true, y_pred, smooth=1, alpha=0.7):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    print(y_true_pos)
    print(y_pred_pos)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    print(true_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    print(false_neg)
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    print(false_pos)
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)

def tversky_loss(y_true, y_pred):
    alpha = 0.5
    beta  = 0.5
    
    ones = K.ones(K.shape(y_true))
    
    p0 = y_pred      # proba that voxels are class i
    p1 = ones-y_pred # proba that voxels are not class i
    print(p1)
    g0 = y_true
    g1 = ones-y_true
    print(g1)
    print(p0*g0)
    num = K.sum(p0*g0, (0))
    print(num)
    den = K.sum(p0*g1,(0))
    print(den)
    
    
    Ncl = K.cast(K.shape(y_true)[-1], 'float32')
    return Ncl-T

def combined_loss(y_true, y_pred):
  return (0.2*K.categorical_crossentropy(y_true, y_pred))+(0.8*tversky_loss(y_true, y_pred))
y_true = [[0,0,1],[1,0,0],[0,1,0]]
y_pred = [[0,1,0],[1,0,0],[0,0,1]]
tversky1 = tversky_loss(np.asarray(y_true),np.asarray(y_pred))


tf.Tensor(
[[1. 0. 1.]
 [0. 1. 1.]
 [1. 1. 0.]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[1. 1. 0.]
 [0. 1. 1.]
 [1. 0. 1.]], shape=(3, 3), dtype=float32)
[[0 0 0]
 [1 0 0]
 [0 0 0]]
tf.Tensor([1 0 0], shape=(3,), dtype=int64)
tf.Tensor([0. 1. 1.], shape=(3,), dtype=float32)


NameError: ignored

In [0]:
PRED_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/PR/KFold LadderNet/"
GT_PATH  = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/4 Class PR/Masks/"
FOLD_PATH = []
files = sorted(os.listdir(PRED_PATH))

# Removing all the .np files from list of folds
for i in range(len(files)):
  if(files[i].split('.')[-1]=="npy"):
    continue
  FOLD_PATH.append(files[i])
print(FOLD_PATH)

['Fold 0', 'Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5', 'Fold 6', 'Fold 7', 'Fold 8']


In [0]:
TrueY = np.empty((0))
PredY = np.empty((0))
SAVE_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/PR/KFold LadderNet/"
patient_no = ['230', '242', '239', '232', '221', '229', '246', '252', '263']
for fold in range(len(FOLD_PATH)):
  print("Fold: " + str(fold))
  test_path = os.listdir(PRED_PATH + FOLD_PATH[fold] + '/Test/')
  for patientID in test_path:
    SM = np.load(PRED_PATH + FOLD_PATH[fold] + '/Test/' + patientID)
    GT = np.load(GT_PATH + 'IHC ' + patientID)
    SM = np.asarray(SM.reshape(-1))
    GT = np.asarray(GT.reshape(-1))
    print(SM.shape)
    print(GT.shape)
    TrueY = np.append(TrueY,SM,axis=0)
    PredY = np.append(PredY,GT,axis=0)
print(PredY.shape)
print(TrueY.shape)

Fold: 0
(38707200,)
(38707200,)
Fold: 1
(27648000,)
(27648000,)
Fold: 2
(27648000,)
(27648000,)
Fold: 3
(27648000,)
(27648000,)
Fold: 4
(27648000,)
(27648000,)
Fold: 5
(27648000,)
(27648000,)
Fold: 6
(27648000,)
(27648000,)
Fold: 7
(27648000,)
(27648000,)
Fold: 8
(27648000,)
(27648000,)
(259891200,)
(259891200,)


In [0]:

prec   = metrics.precision_score(TrueY,PredY,labels=[0,85,170,255],average=None,zero_division=1)
recall = metrics.recall_score(TrueY,PredY,labels=[0,85,170,255],average=None,zero_division=1)
dice = 2*prec*recall/(prec+recall)
print(prec)
print(recall)
print(dice)

[0.98406292 0.1609818  0.51922796 0.84262335]
[0.99521541 0.29920906 0.34337297 0.61287459]
[0.98960775 0.20933581 0.41337504 0.70961617]


**ER**

In [0]:
PRED_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/KFold LadderNet/"
GT_PATH  = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/4 Class ER/Masks/"
FOLD_PATH = []
files = sorted(os.listdir(PRED_PATH))

# Removing all the .np files from list of folds
for i in range(len(files)):
  if(files[i].split('.')[-1]=="npy"):
    continue
  FOLD_PATH.append(files[i])
print(FOLD_PATH)

['Fold 0', 'Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5', 'Fold 6', 'Fold 7', 'Fold 8']


In [0]:
TrueY = np.empty((0))
PredY = np.empty((0))
SAVE_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/PR/KFold LadderNet/"
patient_no = ['230','232','242','263','221','229','239','246','252']
for fold in range(len(FOLD_PATH)):
  print("Fold: " + str(fold))
  test_path = os.listdir(PRED_PATH + FOLD_PATH[fold] + '/Test/')
  for patientID in test_path:
    SM = np.load(PRED_PATH + FOLD_PATH[fold] + '/Test/' + patientID)
    GT = np.load(GT_PATH + 'IHC ' + patientID)
    SM = np.asarray(SM.reshape(-1))
    GT = np.asarray(GT.reshape(-1))
    print(SM.shape)
    print(GT.shape)
    TrueY = np.append(TrueY,SM,axis=0)
    PredY = np.append(PredY,GT,axis=0)
print(PredY.shape)
print(TrueY.shape)

Fold: 0
(27648000,)
(27648000,)
Fold: 1
(27648000,)
(27648000,)
Fold: 2
(27648000,)
(27648000,)
Fold: 3
(27648000,)
(27648000,)
Fold: 4
(27648000,)
(27648000,)
Fold: 5
(27648000,)
(27648000,)
Fold: 6
(27648000,)
(27648000,)
Fold: 7
(27648000,)
(27648000,)
Fold: 8
(27648000,)
(27648000,)
(248832000,)
(248832000,)


In [0]:
prec   = metrics.precision_score(TrueY,PredY,labels=[0,85,170,255],average=None,zero_division=1)
recall = metrics.recall_score(TrueY,PredY,labels=[0,85,170,255],average=None,zero_division=1)
dice = 2*prec*recall/(prec+recall)
print(prec)
print(recall)
print(dice)

[0.92466413 0.29404218 0.33768343 0.75481439]
[0.98678188 0.04579843 0.36055896 0.59182301]
[0.95471366 0.07925286 0.34874648 0.6634548 ]
